In [433]:
import requests as rq
import pandas as pd
import numpy as np
import re
import json
from datetime import datetime

In [434]:
data = rq.get('http://app-sellaci.eu-central-1.elasticbeanstalk.com/').json()

In [435]:
df = pd.DataFrame(data)

In [436]:
# I believe there are a couple of ways to normalize and flatten a nested data. Like using normalize with max_level and transposing it. I am using an approach
# I used to do when dealing with nested Pharmacy and Prescriptions data at work, i.e to make separate entities for each object. To properly do that, I would need a schema
# to describe the attributes of the entites. Also, cool thing about this would be that these separate data frames could be used to populate sql data

# Approach is simple, normalize a dict and make a new dataframe for it. Delete this dict column from parent (To follow RDBMS Rules)

cart_df = pd.json_normalize(df['cart'])
del df['cart']

In [437]:
# Cart has another object paymentIntents
paymentIntents_df = pd.json_normalize(cart_df['paymentIntents'])
# Remove from cart
del cart_df['paymentIntents']
del cart_df['consultationPrice']
cart_df['created_at'] = cart_df['created_at'].apply(lambda x : datetime.strptime(x.split('T')[0],'%Y-%m-%d'))
cart_df['updated_at'] = cart_df['updated_at'].apply(lambda x : datetime.strptime(x.split('T')[0],'%Y-%m-%d'))

In [438]:
# In the parent df, estimatedLength is object with just one key value, that is time
estimatedLength_df = pd.json_normalize(df['estimatedLength'])
# We see that this only contains length in different time units. So, converting it into one unit would be feasible 
for x in df['estimatedLength']:
    if x['unit']=='heure':
        x['value'] = x['value']*3600
        x['unit'] = 'seconds'

    elif x['unit'] == 'minutes':
        x['value'] == x['value']*60
        x['unit'] = 'seconds'
# Replacing original column with consistent data in seconds
df['estimatedLength'] = df['estimatedLength'].apply(lambda x : x['value'])


In [439]:
# Again, parent df has another object initiator with just one value. So replacing
df['intitiator_role'] = pd.json_normalize(df['initiator'])['role']
del df['initiator']

In [440]:
# Snapshot has the most nested objects. Lets normalize this:
snapshot_df = pd.json_normalize(df['snapshot'], max_level= 0)
del df['snapshot']
snapshot_df

,beneficiary,consultation,doctor
0,"{'hasSocialSecurityNumber': True, 'medicalInsu...",{'isPartnershipOfferIncluded': False},"{'isCME': False, 'offer': 'connect'}"
1,"{'hasSocialSecurityNumber': True, 'medicalInsu...",{'isPartnershipOfferIncluded': False},"{'isCME': False, 'offer': 'connect_plus'}"


In [441]:
# Since each of the following is an object and their nested objects only correspond to their specific domain (Like social security number in beneficiary),
#  I am using max_level here for simplicity
beneficiary_df = pd.json_normalize(snapshot_df['beneficiary'], max_level= 3)
consultation_df = pd.json_normalize(snapshot_df['consultation'], max_level= 3)
doctor_df = pd.json_normalize(snapshot_df['doctor'], max_level= 3)

# Later I will show that we can get a patient's SSN quite easily with beneficiary df

In [442]:
# Beneficiary had some empty and NaN values that are of no use for us

beneficiary_df = beneficiary_df.replace(r'^\s*$', np.nan, regex=True)
beneficiary_df = beneficiary_df.dropna(axis=1, how='all')


In [443]:
# For consultation, only one value exists, so making a new column in snapshotdf for this
snapshot_df['consultation_isPartnershipOfferIncluded'] = consultation_df['isPartnershipOfferIncluded']
# Similarly for Doctor values
snapshot_df['doctor_isCME'] = doctor_df['isCME']
snapshot_df['doctor_offer'] = doctor_df['offer']
# We dont need doctordf and consulationdf anymore since snapshot has the values
del doctor_df
del consultation_df

In [444]:
# Simplifying snapshotdf
del snapshot_df['beneficiary']
del snapshot_df['consultation']
del snapshot_df['doctor']
snapshot_df

,consultation_isPartnershipOfferIncluded,doctor_isCME,doctor_offer
0,False,False,connect
1,False,False,connect_plus


In [445]:
# The parent df has a patient phone number object, making a function to return a number
import ast
def returnPhoneNumber(phone):
    phone = ast.literal_eval(phone)
    dialCode = re.findall(r'\d+', phone['dialCode'])[0]

    number = phone['value']
    if(number[0] == '0'):
        number = number[1:]
    number = number.replace(' ', '')
    return '+' + dialCode.strip() + str(number).strip()

# Replacing the object with value using apply
df['patientPhoneNumber'] = df['patientPhoneNumber'].apply(lambda x : returnPhoneNumber(x))


In [446]:
# The parentdf has a healthcenter column which contains either a dict or has nothing in it. It's simply the address. Now in here again, 
# there would be some scenarios where we would want some filtering on City or perhaps the zipcode (I dont know what finess means). In that case we can just 
# add columns for each like healthcenter_address, healthcenter_city, etc. For simplicty and neat look, going with concatenated address.

def returnAddress(address):
    if(isinstance(address, dict)):
        return address['address'] + ', ' + address['city'] + ', ' + address['finess']
    else:
        return None

df['healthCenter'] = df['healthCenter'].apply(lambda x : returnAddress(x))

In [447]:
# Now making a simple patientdf. Lets get their SSN
# Used copy after pandas gave a warning, not sure about why copy fixes it or why is it a warning in the first place

patient_df = df[['patientId','_id','patientPhoneNumber']].copy()
patient_df['SSN'] = beneficiary_df['medicalInsurance.securiteSocialNumber']


In [448]:
patient_df

,patientId,_id,patientPhoneNumber,SSN
0,1209201902,1202019029,+3363223,23232323232311
1,33298320932083920389023,232323231323212098,+33632332,262129ZZZZ05


In [449]:
# The visit ids do not give us any information regarding this dataset for analysis
# I would not drop this in a real data if it were to be given to perhaps the
# development team

del df['visitIds']

In [450]:
# Datetimes were string. Converting them to datetime type

df['estimatedStartDate'] = df['estimatedStartDate'].apply(lambda x : datetime.strptime(x.split('T')[0],'%Y-%m-%d'))
df['createdAt'] = df['createdAt'].apply(lambda x : datetime.strptime(x.split('T')[0],'%Y-%m-%d'))
df['updatedAt'] = df['updatedAt'].apply(lambda x : datetime.strptime(x.split('T')[0],'%Y-%m-%d'))

In [451]:
# This has just unknown values
del df['deviceUsedForBooking']


In [452]:
df

,_id,affiliation,areCotationsAuthorized,canChangeBeneficiary,childId,createdAt,customerCreditHasBeenChecked,customerCreditHasBeenDecreased,details,doctorHasReviewed,doctorId,doctorReminderSent,doctorValidation,estimatedLength,estimatedStartDate,eventId,facturationId,healthCenter,hidePastConsultationsWithOtherPractitioners,isEhpad,isFirstConsultation,isNotConform,isNotConformReason,isSafari,patientAdministrativeArea,patientHasReviewed,patientId,patientPhoneNumber,patientReminderSent,paymentIntentStripeId,preferredMedium,pricingVersion,reminderChecked,reopenCount,speciality,status,transcriptReminderSent,updatedAt,usePaymentIntent,currentTreatments,hasBeenProposedToUpload,hasCurrentTreatments,mediumUsed,patientHasPriorKnowledge,patientIsReady,patientIsReadyDate,termsAcceptedByPatient,uploadWish,intitiator_role
0,1202019029,qare,False,True,,2020-09-27,True,False,"Malade: mal de dos, fièvre, vomissements, covid",False,102901290192091,False,False,10,2021-09-27,23092039023,121212,"39bis Avenue de Champeret, Charenton, 2323232",False,False,False,False,[],False,92,False,1209201902,+3363223,False,,video,6,False,0,general-practician,assigned,False,2020-09-27,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,patient
1,232323231323212098,qare,True,True,23983099839023,2021-09-27,True,False,"Trouble du sommeil, tension, douleurs au dos",False,2308863920973287839723,False,False,3600,2021-09-27,736628369276382638722,5910164Z23,None,False,False,False,False,[],False,9z5,False,33298320932083920389023,+33632332,False,30F24wbSpLuC0Xka8T6j,video,6,False,0,general-practician,assigned,False,2021-09-27,True,"Antidépresseur, anxiolitique",True,Y,android - VOG-L29,True,True,2021-09-27T21:25:53.140Z,True,N,patient


In [453]:
snapshot_df

,consultation_isPartnershipOfferIncluded,doctor_isCME,doctor_offer
0,False,False,connect
1,False,False,connect_plus


In [454]:
beneficiary_df

,hasSocialSecurityNumber,medicalInsurance.privateInsurance.amcId,medicalInsurance.privateInsurance.medicalRecordId,medicalInsurance.privateInsurance.scanResult,medicalInsurance.privateInsurance.verifiedAt,medicalInsurance.privateInsurance.verifiedStatus,medicalInsurance.securiteSocialNumber,medicalInsurance.verifiedAt,medicalInsurance.verifiedStatus
0,True,2323242,232314234234,AMC#1#775685399#100#0103818949#MU*/,2021-09-18T07:36:49.537Z,SUCCESS,23232323232311,2021-09-18T07:36:49.495Z,SUCCESS
1,True,NaN,NaN,NaN,NaN,NaN,262129ZZZZ05,NaN,NaN


In [455]:
cart_df

,act,actId,created_at,currency,customerId,id,isChargeable,isImmediateCharge,isRequirable,majF,majMCG,majMEG,majMN35,majMN40,majMPA,price,priceMin,quotedPrice,status,tokenId,updated_at,usePaymentIntent,explanation
0,consultation,9423320,2020-09-26,EUR,232323,2323232,False,False,False,0,5,0,35,0,0,25,25,0,required,Hs3330qZR,2020-09-26,True,NaN
1,consultation,232131,2021-09-27,EUR,232312,232311,True,False,False,0,5,0,35,0,0,25,25,25,required,pWI_S,2021-09-27,True,always


In [456]:
beneficiary_df

,hasSocialSecurityNumber,medicalInsurance.privateInsurance.amcId,medicalInsurance.privateInsurance.medicalRecordId,medicalInsurance.privateInsurance.scanResult,medicalInsurance.privateInsurance.verifiedAt,medicalInsurance.privateInsurance.verifiedStatus,medicalInsurance.securiteSocialNumber,medicalInsurance.verifiedAt,medicalInsurance.verifiedStatus
0,True,2323242,232314234234,AMC#1#775685399#100#0103818949#MU*/,2021-09-18T07:36:49.537Z,SUCCESS,23232323232311,2021-09-18T07:36:49.495Z,SUCCESS
1,True,NaN,NaN,NaN,NaN,NaN,262129ZZZZ05,NaN,NaN
